In [1]:
import PyROA
import os
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import os
import glob
import scienceplots
import sys
sys.path.insert(0, os.path.abspath('..'))
import rm_utils
sys.path.insert(0, os.path.abspath('../..'))
import utils
np.random.seed(2) 
torch.manual_seed(2)
%load_ext autoreload

In [4]:
%autoreload

In [2]:
lcs = utils.get_data('../../datasets/ZTF_rm_segments/NGC5548', start_col=0,sep=' ', norm=False, shuffle=False, keep_missing=True)

found 1 for band='epoch5'
found 2 for band='epoch2'
found 2 for band='epoch3'
found 2 for band='epoch4'
found 2 for band='epoch1'
found 1 for band='epoch6'
found 3 for band='epoch0'
max time:  59353.164
created union_tp attribute of length 3500
dataset created, lcs.dataset.shape=(1, 7, 238, 3)
train size: 0, valid size: 0, test size: 1


In [3]:
lcs.valid_files_df

,epoch5,epoch2,epoch3,epoch4,epoch1,epoch6,epoch0
MCG+08-11-011,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...,../../datasets/ZTF_rm_segments/MCG+08-11-011/e...


In [29]:
epoch_dirs = glob.glob('../../datasets/ZTF_rm_segments/NGC5548/*/')
epoch_dirs.sort()
print(epoch_dirs, len(epoch_dirs))

filters = ["g","r"] 
objName = "MCG+08-11-011"
psi_types = ["LogGaussian"]


default = [[0.0, 20.0],[0.0, 100.0], [-50.0, 50.0], [0.01, 10.0], [0.0, 10.0]]
            # rms      # mean       # lag        # width of window  # extra error 
priors = [[0.0, 20.0], [0.0, 100.0], [-50.0, 50.0], [0.01, 10.0], [0.0, 10.0]]

inc_window = [[0.0, 20.0], [0.0, 100.0], [-50.0, 50.0], [10,30], [0.0, 10.0]]
 


['../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch0/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch1/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch2/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch3/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch4/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch5/', '../../datasets/ZTF_rm_segments/MCG+08-11-011/epoch6/'] 7


In [ ]:
0,1,2,3 work for all bands
4 only for g,r